# Script to get the jacobian matrix which under the .ipynb format

In [3]:
import sympy as sym
import numpy as np

In [6]:
# Direct state vector
x, y, z, w, l, h, v, a, yaw, sigma, lf_r, w_r = sym.symbols("x y z w h l v a yaw sigma lf_r w_r")
next_x, next_y, next_z, next_w, next_h, next_l, next_v, next_a, next_yaw, next_sigma = sym.symbols("next_x next_y next_z next_w next_h next_l next_v next_a next_yaw next_sigma")
dt = sym.symbols("dt")

In [12]:
# tmp state vector
beta, ry_rate, lf, lr = sym.symbols("beta ry_rate lf lr")
lf, lr = l * w_r * lf_r, l * w_r * (1 - lf_r)
beta = sym.atan(lr / (l * w_r) * sym.tan(sigma))

In [13]:
# constant state vector
next_z, next_w, next_h, next_l, next_a, next_sigma, next_v = z, w, h, l, 0, sigma, v

In [14]:
# heading yaw
ry_rate = v / lr * sym.sin(beta)
next_yaw = yaw + ry_rate * dt

In [15]:
vt, vyawt, ita, t = sym.symbols("vt vyawt ita t")

# warp raw transition time from [(t-1)*dt, t*dt] to [0, dt], for quick solution
vt = v
vyawt = beta + yaw + ry_rate * (ita)
next_x = x + sym.integrate(vt * sym.cos(vyawt), (ita, 0, dt))
next_y = y + sym.integrate(vt * sym.sin(vyawt), (ita, 0, dt))

# Analytical solutions for displacements can also be obtained

In [16]:
next_x

x + Piecewise((dt*v*cos(yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r)), Eq(v, 0) | (Eq(v, 0) & Eq(tan(sigma), 0))), (dt*v*cos(yaw), Eq(sigma, 0)), (-h*w_r*sqrt(lf_r**2*tan(sigma)**2 - 2*lf_r*tan(sigma)**2 + tan(sigma)**2 + 1)*sin(yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r))/tan(sigma) + h*w_r*sqrt(lf_r**2*tan(sigma)**2 - 2*lf_r*tan(sigma)**2 + tan(sigma)**2 + 1)*sin(dt*v*tan(sigma)/(h*w_r*sqrt((1 - lf_r)**2*tan(sigma)**2 + 1)) + yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r))/tan(sigma), True))

In [17]:
next_y

y + Piecewise((dt*v*sin(yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r)), Eq(v, 0) | (Eq(v, 0) & Eq(tan(sigma), 0))), (dt*v*sin(yaw), Eq(sigma, 0)), (h*w_r*sqrt(lf_r**2*tan(sigma)**2 - 2*lf_r*tan(sigma)**2 + tan(sigma)**2 + 1)*cos(yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r))/tan(sigma) - h*w_r*sqrt(lf_r**2*tan(sigma)**2 - 2*lf_r*tan(sigma)**2 + tan(sigma)**2 + 1)*cos(dt*v*tan(sigma)/(h*w_r*sqrt((1 - lf_r)**2*tan(sigma)**2 + 1)) + yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r))/tan(sigma), True))

In [18]:
next_yaw

dt*v*tan(sigma)/(h*w_r*sqrt((1 - lf_r)**2*tan(sigma)**2 + 1)) + yaw

In [20]:
funcs = sym.Matrix([next_x, next_y, next_z, next_w, next_l, next_h, next_v, next_a, next_yaw, next_sigma])
args = sym.Matrix([x, y, z, w, l, h, v, a, yaw, sigma])
res = funcs.jacobian(args)

In [22]:
res

Matrix([
[1, 0, 0, 0, Piecewise((0, (Eq(sigma, 0) | Eq(v, 0)) & (Eq(sigma, 0) | Eq(v, 0) | Eq(tan(sigma), 0))), (-dt*v*sqrt(lf_r**2*tan(sigma)**2 - 2*lf_r*tan(sigma)**2 + tan(sigma)**2 + 1)*cos(dt*v*tan(sigma)/(h*w_r*sqrt((1 - lf_r)**2*tan(sigma)**2 + 1)) + yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r))/(h*sqrt((1 - lf_r)**2*tan(sigma)**2 + 1)) - w_r*sqrt(lf_r**2*tan(sigma)**2 - 2*lf_r*tan(sigma)**2 + tan(sigma)**2 + 1)*sin(yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r))/tan(sigma) + w_r*sqrt(lf_r**2*tan(sigma)**2 - 2*lf_r*tan(sigma)**2 + tan(sigma)**2 + 1)*sin(dt*v*tan(sigma)/(h*w_r*sqrt((1 - lf_r)**2*tan(sigma)**2 + 1)) + yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r))/tan(sigma), True)), 0, Piecewise((dt*cos(yaw + atan((1 - lf_r)*tan(sigma)) + pi*floor((sigma - pi/2)/pi)*sign(1 - lf_r)), Eq(v, 0) | (Eq(v, 0) & Eq(tan(sigma), 0))), (dt*cos(yaw), Eq(sigma, 0)), (dt*sqrt(lf_r**2*tan(sigma)*

In [41]:
mea_funcs = sym.Matrix([x, y, z, w, l, h, v * sym.cos(yaw), v * sym.sin(yaw), yaw])
mea_res = mea_funcs.jacobian(args)

In [56]:
mea_res, mea_res

(Matrix([
 [1, 0, 0, 0, 0, 0,        0, 0,           0, 0],
 [0, 1, 0, 0, 0, 0,        0, 0,           0, 0],
 [0, 0, 1, 0, 0, 0,        0, 0,           0, 0],
 [0, 0, 0, 1, 0, 0,        0, 0,           0, 0],
 [0, 0, 0, 0, 1, 0,        0, 0,           0, 0],
 [0, 0, 0, 0, 0, 1,        0, 0,           0, 0],
 [0, 0, 0, 0, 0, 0, cos(yaw), 0, -v*sin(yaw), 0],
 [0, 0, 0, 0, 0, 0, sin(yaw), 0,  v*cos(yaw), 0],
 [0, 0, 0, 0, 0, 0,        0, 0,           1, 0]]),
 Matrix([
 [1, 0, 0, 0, 0, 0,        0, 0,           0, 0],
 [0, 1, 0, 0, 0, 0,        0, 0,           0, 0],
 [0, 0, 1, 0, 0, 0,        0, 0,           0, 0],
 [0, 0, 0, 1, 0, 0,        0, 0,           0, 0],
 [0, 0, 0, 0, 1, 0,        0, 0,           0, 0],
 [0, 0, 0, 0, 0, 1,        0, 0,           0, 0],
 [0, 0, 0, 0, 0, 0, cos(yaw), 0, -v*sin(yaw), 0],
 [0, 0, 0, 0, 0, 0, sin(yaw), 0,  v*cos(yaw), 0],
 [0, 0, 0, 0, 0, 0,        0, 0,           1, 0]]))

In [43]:
yaw_sin, yaw_cos, dt, a, v = np.sin(1), np.cos(1), 1, 1, 1
theta, omega = 1, 1
ry_rate_inv, ry_rate_inv_square, ry_rate_inv_cube = 1 / omega, 1 / (omega * omega), 1 / (omega * omega * omega)
next_v, next_ry = v + a * dt, theta + omega * dt
next_yaw_sin, next_yaw_cos = np.sin(next_ry), np.cos(next_ry)

In [49]:
F = np.mat([[1, 0, 0, 0, 0, 0,  -ry_rate_inv*(yaw_sin-next_yaw_sin),   -ry_rate_inv_square*(yaw_cos-next_yaw_cos)+ry_rate_inv*dt*next_yaw_sin,        ry_rate_inv_square*a*(yaw_sin-next_yaw_sin)+ry_rate_inv*(next_v*next_yaw_cos-v*yaw_cos),  ry_rate_inv_cube*2*a*(yaw_cos-next_yaw_cos)+ry_rate_inv_square*(v*yaw_sin-v*next_yaw_sin-2*a*dt*next_yaw_sin)+ry_rate_inv*dt*next_v*next_yaw_cos ],
                        [0, 1, 0, 0, 0, 0,   ry_rate_inv*(yaw_cos-next_yaw_cos),   -ry_rate_inv_square*(yaw_sin-next_yaw_sin)-ry_rate_inv*dt*next_yaw_cos,        ry_rate_inv_square*a*(-yaw_cos+next_yaw_cos)+ry_rate_inv*(next_v*next_yaw_sin-v*yaw_sin), ry_rate_inv_cube*2*a*(yaw_sin-next_yaw_sin)+ry_rate_inv_square*(v*next_yaw_cos-v*yaw_cos+2*a*dt*next_yaw_cos)+ry_rate_inv*dt*next_v*next_yaw_sin ],
                        [0, 0, 1, 0, 0, 0,           0,                 0,                            0,  0],
                        [0, 0, 0, 1, 0, 0,           0,                 0,                            0,  0],
                        [0, 0, 0, 0, 1, 0,           0,                 0,                            0,  0],
                        [0, 0, 0, 0, 0, 1,           0,                 0,                            0,  0],
                        [0, 0, 0, 0, 0, 0,           1,                dt,                            0,  0],
                        [0, 0, 0, 0, 0, 0,           0,                 1,                            0,  0],
                        [0, 0, 0, 0, 0, 0,           0,                 0,                            1, dt],
                        [0, 0, 0, 0, 0, 0,           0,                 0,                            0,  1]])
                   

In [54]:
displacement = v * dt + a * dt ** 2 / 2
F = np.mat([[1, 0, 0, 0, 0, 0,  dt*yaw_cos,   dt**2*yaw_cos/2,        -displacement*yaw_sin,  0],
            [0, 1, 0, 0, 0, 0,  dt*yaw_sin,   dt**2*yaw_sin/2,         displacement*yaw_cos,  0],
            [0, 0, 1, 0, 0, 0,           0,                 0,                            0,  0],
            [0, 0, 0, 1, 0, 0,           0,                 0,                            0,  0],
            [0, 0, 0, 0, 1, 0,           0,                 0,                            0,  0],
            [0, 0, 0, 0, 0, 1,           0,                 0,                            0,  0],
            [0, 0, 0, 0, 0, 0,           1,                dt,                            0,  0],
            [0, 0, 0, 0, 0, 0,           0,                 1,                            0,  0],
            [0, 0, 0, 0, 0, 0,           0,                 0,                            1, dt],
            [0, 0, 0, 0, 0, 0,           0,                 0,                            0,  1]])

In [30]:
res.evalf(subs ={'dt':1, 'v':1, 'a':1, 'ry_rate':1, 'yaw':1})

Matrix([
[1.0,   0,   0,   0,   0,   0, 0.0678264420177852, -0.0471517155896004,  -1.44042242098021, -0.805816683932869],
[  0, 1.0,   0,   0,   0,   0,  0.956449142415282,   0.483973278564928, 0.0206747264281848, -0.105800845893774],
[  0,   0, 1.0,   0,   0,   0,                  0,                   0,                  0,                  0],
[  0,   0,   0, 1.0,   0,   0,                  0,                   0,                  0,                  0],
[  0,   0,   0,   0, 1.0,   0,                  0,                   0,                  0,                  0],
[  0,   0,   0,   0,   0, 1.0,                  0,                   0,                  0,                  0],
[  0,   0,   0,   0,   0,   0,                1.0,                 1.0,                  0,                  0],
[  0,   0,   0,   0,   0,   0,                  0,                 1.0,                  0,                  0],
[  0,   0,   0,   0,   0,   0,                  0,                   0,                

In [55]:
F - res.evalf(subs ={'dt':1, 'v':1, 'a':1, 'ry_rate':0, 'yaw':1})

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1.11022302462516e-16, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,                    0, 0]])

In [51]:
F = np.mat([[1, 0, 0, 0, 0, 0,  -ry_rate_inv*(yaw_sin-next_yaw_sin),   -ry_rate_inv_square*(yaw_cos-next_yaw_cos)+ry_rate_inv*dt*next_yaw_sin,        ry_rate_inv_square*a*(yaw_sin-next_yaw_sin)+ry_rate_inv*(next_v*next_yaw_cos-v*yaw_cos),  ry_rate_inv_cube*2*a*(yaw_cos-next_yaw_cos)+ry_rate_inv_square*(v*yaw_sin-v*next_yaw_sin-2*a*dt*next_yaw_sin)+ry_rate_inv*dt*next_v*next_yaw_cos ],
                        [0, 1, 0, 0, 0, 0,   ry_rate_inv*(yaw_cos-next_yaw_cos),   -ry_rate_inv_square*(yaw_sin-next_yaw_sin)-ry_rate_inv*dt*next_yaw_cos,        ry_rate_inv_square*a*(-yaw_cos+next_yaw_cos)+ry_rate_inv*(next_v*next_yaw_sin-v*yaw_sin), ry_rate_inv_cube*2*a*(yaw_sin-next_yaw_sin)+ry_rate_inv_square*(v*next_yaw_cos-v*yaw_cos+2*a*dt*next_yaw_cos)+ry_rate_inv*dt*next_v*next_yaw_sin ],
                        [0, 0, 1, 0, 0, 0,           0,                 0,                            0,  0],
                        [0, 0, 0, 1, 0, 0,           0,                 0,                            0,  0],
                        [0, 0, 0, 0, 1, 0,           0,                 0,                            0,  0],
                        [0, 0, 0, 0, 0, 1,           0,                 0,                            0,  0],
                        [0, 0, 0, 0, 0, 0,           1,                dt,                            0,  0],
                        [0, 0, 0, 0, 0, 0,           0,                 1,                            0,  0],
                        [0, 0, 0, 0, 0, 0,           0,                 0,                            1, dt],
                        [0, 0, 0, 0, 0, 0,           0,                 0,                            0,  1]])
                                

In [8]:
mea_funcs = sym.Matrix([x, y, z, w, l, h, yaw])
mea_res = mea_funcs.jacobian(args)

In [9]:
mea_res

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])